In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, RobustScaler
from collections import Counter
from Model_Utils import Sequence_Model, Classify_Model, Embedding_Model, HeadProjection_Model, Model_Combine
from Dataset_Utils import *
from General_Utils import *
from Loss_Utils import *
from torch.optim.lr_scheduler import ReduceLROnPlateau
import joblib
from scipy import stats
#torch.autograd.set_detect_anomaly(True)

# 14, 6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5, 16
SUBJECT_ID_TEST = [6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5]
SUBJECT_ID_TEST = ['S'+str(x) for x in SUBJECT_ID_TEST]
SUBJECT_ID_TEST = 'S14'

# # 'GM1', 'EK1', 'NM1', 'RY1', 'KSG1', 'AD1', 'NM3', 'SJ1', 'BK1', 'RY2', 'GM2', 'MT1', 'NM2'
# SUBJECT_ID_TEST = 'AD1' # SJ1

USE_LARS = True
OPTIM = 'Adam'
LR = 0.003

MARGIN = 1
SEQ_DIM = 128
INPUT_FT = 60
EMBEDDING_HIDDEN = [256, 512]
PROJECTION_OUT = None #128 # None or int
CLASSIFY_HIDDEN = [-1] # [64] # list or None
DROPOUT = 0.2

MAX_EPOCH = 50
LOOK_BEFORE = 3
INTERNAL_SAMPLE = 200

MODEL_NAME = 'Euclid_CLS_512_LARS_Adam'
SAVE_MODEL_DIR = 'Output'
NAME_DATASET = 'WESAD'

##### READ DATASET #####
if NAME_DATASET == 'WESAD':
    DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/WESAD'
    data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_groups_1.npy')
    data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_ground_truth_1.npy')
    data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_WRIST_stats_feats_1.npy')
else:
    DATA_DIR = '/home/nvtu/PhD_Work/StressDetection/DATA/MyDataset/AffectiveROAD_Data/Database'
    NAME_DATASET = 'AffectiveROAD'
    data_group = np.load(f'{DATA_DIR}/{NAME_DATASET}_groups_1.npy')
    data_gt = np.load(f'{DATA_DIR}/{NAME_DATASET}_ground_truth_1.npy')
    data_ft = np.load(f'{DATA_DIR}/{NAME_DATASET}_stats_feats_1.npy')
    indices = np.where(data_gt >= 0)[0]
    data_ft = data_ft[indices]
    data_group = data_group[indices]
    data_gt = data_gt[indices]

# Create dataframe for dataset
column_values = [f'f{x}' for x in range(data_ft.shape[1])]
data_full = pd.DataFrame(data = data_ft,  
                         columns = column_values)
data_full['subject_id'] = data_group
data_full['label'] = data_gt
list_subject_id = np.unique(data_full['subject_id']).tolist()

##### TRAIN / VAL / TEST #####
subject_id_test = SUBJECT_ID_TEST
data_train_val = data_full[data_full.subject_id != subject_id_test]
data_test = data_full[data_full.subject_id == subject_id_test]
list_id = list(set(data_train_val.subject_id))
list_id.sort()
subject_id_validate = random.Random(1509+int(subject_id_test[1:])+88).choices(list_id,k=1)[0]
#subject_id_validate = 'S8'
data_train = data_train_val[data_train_val.subject_id != subject_id_validate]
data_validate = data_train_val[data_train_val.subject_id == subject_id_validate]
ft_names = data_train.columns.tolist()

print(f"Training Model for {subject_id_test} and validate on {subject_id_validate}")

#temp = Counter(data_train.iloc[:,-1].tolist())
#print(temp)

# Scaler Data
X_train = data_train.iloc[:,:-2].to_numpy()
group_train = data_train.iloc[:,-2].to_numpy()
y_train = data_train.iloc[:,-1].to_numpy()
X_test = data_test.iloc[:,:-2].to_numpy()
group_test = data_test.iloc[:,-2].to_numpy()
y_test = data_test.iloc[:,-1].to_numpy()
X_validate = data_validate.iloc[:,:-2].to_numpy()
group_validate = data_validate.iloc[:,-2].to_numpy()
y_validate = data_validate.iloc[:,-1].to_numpy()

X_train = X_train.astype('float64')
X_validate = X_validate.astype('float64')
X_test = X_test.astype('float64')

y_train = y_train.astype('float64')
y_validate = y_validate.astype('float64')
y_test = y_test.astype('float64')

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validate = scaler.transform(X_validate)
X_test = scaler.transform(X_test)
#joblib.dump(scaler, f'{SAVE_MODEL_DIR}/{NAME_DATASET}/Model/StandardScaler_{subject_id_test}.joblib')

# Create Dataframe
df_train = pd.DataFrame(data = X_train, columns = ft_names[:-2])
df_train['subject_id'] = group_train
df_train['label'] = y_train

df_validate = pd.DataFrame(data = X_validate, columns = ft_names[:-2])
df_validate['subject_id'] = group_validate
df_validate['label'] = y_validate

df_test= pd.DataFrame(data = X_test, columns = ft_names[:-2])
df_test['subject_id'] = group_test
df_test['label'] = y_test

Training Model for S14 and validate on S9


In [3]:
validate_dataset_embed = EmbDataset(df_validate)
validate_dataloader_embed = make_EmbDataLoader(validate_dataset_embed, batch_size=2, shuffle=False)

In [4]:
df_validate.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f52,f53,f54,f55,f56,f57,f58,f59,subject_id,label
0,0.084262,-0.402805,-0.208260,-0.239780,-0.158864,0.152016,-0.636399,-0.723279,-0.753620,-0.21681,...,0.423549,-0.452453,0.426066,-0.449083,-0.244755,-0.439654,-0.280814,-0.467668,S9,0.0
1,0.042044,-0.555111,-0.189701,-0.252628,-0.143778,0.241302,-0.679626,-0.763739,-0.815771,-0.21681,...,0.428999,-0.452457,0.426064,-0.451794,-0.244799,-0.440175,-0.280849,-0.468716,S9,0.0
2,0.063174,-0.557960,-0.202321,-0.254617,-0.134279,0.259216,-0.679626,-0.763739,-0.815771,-0.21681,...,0.435038,-0.452465,0.426060,-0.455167,-0.244819,-0.442701,-0.280908,-0.470521,S9,0.0
3,0.033235,-0.567356,-0.182903,-0.252885,-0.142263,0.244829,-0.679626,-0.741486,-0.832197,-0.21681,...,0.431987,-0.452460,0.426061,-0.452361,-0.244818,-0.441616,-0.280867,-0.469267,S9,0.0
4,0.025635,-0.544686,-0.175426,-0.243760,-0.159953,0.239533,-0.636399,-0.700736,-0.769602,-0.21681,...,0.443693,-0.452476,0.426057,-0.458380,-0.244821,-0.443547,-0.280922,-0.470958,S9,0.0


In [5]:
a, b = next(iter(validate_dataloader_embed))

In [6]:
a

tensor([[ 0.0843, -0.4028, -0.2083, -0.2398, -0.1589,  0.1520, -0.6364, -0.7233,
         -0.7536, -0.2168, -0.2207, -0.0558,  0.1025, -0.0795, -0.2442,  0.5074,
         -0.2201, -0.2534, -0.4827,  1.1039, -0.2133, -0.2540, -0.1979, -0.0802,
          0.9436, -0.6417, -0.4469, -0.4677, -0.8779,  0.0417,  0.2269, -0.4103,
         -0.4184,  0.0377, -0.6417, -0.4796, -0.6412, -0.6543, -0.5218, -0.1311,
         -0.5485,  0.3815, -0.2764, -0.6131, -0.1531, -0.4306,  0.2607, -0.4034,
         -0.5031, -0.5485,  0.1206, -0.5521,  0.4235, -0.4525,  0.4261, -0.4491,
         -0.2448, -0.4397, -0.2808, -0.4677],
        [ 0.0420, -0.5551, -0.1897, -0.2526, -0.1438,  0.2413, -0.6796, -0.7637,
         -0.8158, -0.2168, -0.2383,  0.0744,  0.1879, -0.0793, -0.2255,  0.5095,
         -0.2376, -0.2538, -0.4573, -0.8411, -0.2133, -0.3642, -0.2832, -0.1450,
          0.4765, -0.6419, -0.4489, -0.4687, -0.8807,  0.0417,  0.1961, -0.4140,
         -0.4217,  0.0620, -0.6420, -0.4803, -0.6412, -0.6557, 

In [7]:
b

tensor([0., 0.])

In [4]:
scaler.var_

array([1.09943190e+02, 5.14741946e+01, 2.18883095e+04, 1.05059296e+05,
       1.83432520e+02, 2.92378026e+00, 6.95806452e+02, 3.87643737e+02,
       1.82590848e+02, 5.33039201e+06, 1.43445357e+05, 2.99825364e-04,
       1.44043042e-03, 1.33659821e+03, 3.79557109e-02, 2.56436559e-02,
       7.30677087e+04, 1.40080942e+05, 2.19675923e-02, 1.30678450e-05,
       2.48847839e-04, 1.56271778e-02, 4.70061265e-02, 2.25893961e+01,
       2.16067643e-01, 5.90948987e+00, 1.83456505e-02, 2.10852517e-03,
       5.64294518e-01, 1.82855211e+03, 4.51592610e+03, 3.95734791e+00,
       1.89084947e+01, 1.83724966e+03, 5.90946520e+00, 2.05697695e-02,
       4.81998477e+00, 6.81183910e+00, 3.41316350e-01, 1.27865391e-06,
       2.56900978e-02, 4.74773872e-02, 5.25892202e+02, 5.32193603e+00,
       3.12648400e-09, 3.53963729e-04, 9.79407567e-09, 1.83384306e-04,
       3.81346924e-03, 2.56900978e-02, 1.54511594e-03, 2.10634734e-03,
       7.81849183e-04, 5.70927486e-03, 3.07104350e-02, 3.45686267e-03,
      

In [5]:
data_train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f52,f53,f54,f55,f56,f57,f58,f59,subject_id,label
0,71.001875,11.594887,853.642086,154.521049,2.339071,0.333258,109.0,78.417266,94.244604,1125.0,...,-0.002344,0.002462,-0.018732,0.003601,21.886141,0.890333,0.000010,0.003222,S14,0
1,71.275764,11.993190,852.068345,156.208126,2.194700,0.313355,110.0,79.136691,94.964029,1125.0,...,-0.001870,0.002883,-0.009692,0.002393,21.886140,0.801697,0.000006,0.002370,S14,0
2,70.995376,11.993262,854.428957,158.174694,2.008055,0.290720,110.0,79.136691,94.964029,1125.0,...,-0.001849,0.002447,-0.009703,0.002347,21.886139,0.813464,0.000006,0.002419,S14,0
3,70.929045,12.084673,854.991007,159.511131,1.910564,0.295406,109.0,78.417266,94.244604,1125.0,...,-0.001749,0.003175,-0.009677,0.002320,21.886140,0.762883,0.000005,0.002215,S14,0
4,71.047971,12.048325,853.866906,159.098395,1.924567,0.293828,109.0,78.417266,94.964029,1125.0,...,-0.001919,0.001426,-0.009659,0.002218,21.886143,0.726163,0.000005,0.002130,S14,0


In [7]:
data_train['f9'].mean()

1768.4307213451418

In [8]:
data_train['f9'].std()

2308.8012175492113

In [13]:
modules = [] 
emb_size = [128]
for idx, size in enumerate(emb_size):
    if idx == 0:
        modules.append(nn.Linear(60, emb_size[idx]))
    else:
        modules.append(nn.Linear(emb_size[idx-1], emb_size[idx]))
    modules.append(nn.BatchNorm1d(num_features=emb_size[idx]))
    modules.append(nn.ReLU())
    modules.append(nn.Dropout(0.2))   

# Add for Res connect
#modules.append(nn.Linear(self.emb_size[-1], self.input_size))
#modules.append(nn.BatchNorm1d(num_features=self.input_size))
#modules.append(nn.ReLU())
#modules.append(nn.Dropout(self.do))    
emb = nn.Sequential(*modules)

In [17]:
emb[0].weight

Parameter containing:
tensor([[ 0.0988, -0.0020, -0.0154,  ...,  0.0274,  0.1140, -0.0258],
        [-0.0066,  0.0913, -0.1289,  ..., -0.0730,  0.1078, -0.1040],
        [-0.0611, -0.0070, -0.1198,  ..., -0.0228, -0.0460, -0.0347],
        ...,
        [ 0.0742, -0.0556,  0.0104,  ...,  0.0198, -0.0373,  0.0250],
        [-0.0158,  0.0866, -0.0060,  ..., -0.1018,  0.0731,  0.0742],
        [ 0.0228, -0.0764, -0.0969,  ..., -0.0252, -0.0199,  0.0934]],
       requires_grad=True)

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, RobustScaler
from collections import Counter
from Model_Utils import Sequence_Model, Classify_Model, Embedding_Model, HeadProjection_Model
from Dataset_Utils import *
from General_Utils import *
from torch.optim.lr_scheduler import ReduceLROnPlateau
import joblib
from scipy import stats

# 14, 6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5, 16
SUBJECT_ID_TEST = [6, 17, 3, 2, 13, 9, 10, 15, 8, 7, 11, 4, 5]
SUBJECT_ID_TEST = ['S'+str(x) for x in SUBJECT_ID_TEST]
SUBJECT_ID_TEST = 'S9'

# # 'GM1', 'EK1', 'NM1', 'RY1', 'KSG1', 'AD1', 'NM3', 'SJ1', 'BK1', 'RY2', 'GM2', 'MT1', 'NM2'
# SUBJECT_ID_TEST = 'AD1' # SJ1

MARGIN = 1
SEQ_DIM = 128
INPUT_FT = 60
EMBEDDING_HIDDEN = [60]
PROJECTION_OUT = 32
CLASSIFY_HIDDEN = [64]
DROPOUT = 0.2
LR = 0.003
MAX_EPOCH = 50
LOOK_BEFORE = 3
INTERNAL_SAMPLE = 200

MODEL_NAME = 'Emb_Euclid_sample_cross_internal_Projection'
SAVE_MODEL_DIR = 'Output'
NAME_DATASET = 'WESAD'


In [2]:
Con_Mod = Embedding_Model(input_size=INPUT_FT, dropout=DROPOUT, emb_size=EMBEDDING_HIDDEN).to(device)
Pro_Mod = HeadProjection_Model(input_size=EMBEDDING_HIDDEN[-1], output_size=PROJECTION_OUT).to(device)

modelCheckpoint = torch.load(f"{SAVE_MODEL_DIR}/{NAME_DATASET}/Model/{MODEL_NAME}_{SUBJECT_ID_TEST}.pth.tar")
Con_Mod.load_state_dict(modelCheckpoint['model_con_state_dict'])
Con_Mod.eval()
Pro_Mod.load_state_dict(modelCheckpoint['model_pro_state_dict'])
Pro_Mod.eval()

HeadProjection_Model(
  (hd1): Linear(in_features=60, out_features=60, bias=True)
  (rl1): ReLU()
  (hd2): Linear(in_features=60, out_features=32, bias=True)
)

In [3]:
Pro_Mod.hd1.weight

Parameter containing:
tensor([[-0.0858,  0.0980, -0.1024,  ...,  0.0130,  0.0648,  0.1077],
        [ 0.0520, -0.0825,  0.1032,  ...,  0.0158,  0.0317,  0.0155],
        [-0.0750, -0.0975,  0.0795,  ..., -0.0446, -0.0503, -0.0287],
        ...,
        [ 0.0251,  0.0366,  0.0377,  ..., -0.0401,  0.0422, -0.0394],
        [ 0.0049, -0.0798, -0.0865,  ...,  0.0496, -0.0921, -0.0768],
        [ 0.0981, -0.0390, -0.0083,  ...,  0.0266,  0.0370,  0.0711]],
       device='cuda:0', requires_grad=True)

In [4]:
Pro_Mod.hd1.bias

Parameter containing:
tensor([-0.0934, -0.1215,  0.2051, -0.0744,  0.2129,  0.2045,  0.0590,  0.2637,
         0.2138,  0.0058,  0.0788,  0.1497, -0.0710, -0.0327, -0.0213,  0.1931,
        -0.0747, -0.1069,  0.0922, -0.0311, -0.1023,  0.1935,  0.1745, -0.1163,
         0.2568,  0.0939, -0.0642,  0.0359,  0.1852,  0.2701, -0.1405,  0.2189,
         0.2580,  0.2921, -0.0579,  0.2543, -0.0737, -0.0364,  0.2275, -0.1183,
        -0.0726, -0.1142, -0.1225, -0.0979,  0.0906,  0.0431, -0.1285, -0.0323,
        -0.1574,  0.2514,  0.0058,  0.0200, -0.0503, -0.1350,  0.1947,  0.1999,
         0.1651,  0.1295, -0.1420,  0.1725], device='cuda:0',
       requires_grad=True)

In [5]:
Pro_Mod.hd2.bias

Parameter containing:
tensor([-0.1596,  0.0928,  0.1766,  0.0669, -0.1054, -0.1328,  0.0818, -0.0764,
         0.0954,  0.0787, -0.0842,  0.0649, -0.0980, -0.1510,  0.0510, -0.0521,
        -0.0863,  0.0436, -0.1306, -0.1463,  0.0967, -0.0999,  0.0709,  0.1425,
         0.1492,  0.1079, -0.1419, -0.0056, -0.1357, -0.0456,  0.0960,  0.1696],
       device='cuda:0', requires_grad=True)

In [6]:
Pro_Mod.hd2.weight

Parameter containing:
tensor([[ 0.0513, -0.0392, -0.0430,  ..., -0.0796, -0.0425,  0.0383],
        [ 0.0412, -0.0211,  0.0820,  ...,  0.0713,  0.1204,  0.0695],
        [ 0.0698,  0.0704,  0.0572,  ...,  0.0843,  0.0856,  0.0831],
        ...,
        [-0.1059,  0.0759, -0.0796,  ..., -0.0353, -0.1383, -0.0668],
        [ 0.0537, -0.0231,  0.0292,  ...,  0.0964, -0.0417,  0.0113],
        [-0.0549,  0.1594,  0.0441,  ..., -0.0246, -0.0376, -0.0178]],
       device='cuda:0', requires_grad=True)

In [7]:
a = torch.rand((2, 60)).cuda()

In [8]:
b = Pro_Mod(a)

In [10]:
b

tensor([[-0.1397,  0.2591,  0.1535,  0.1633, -0.1482, -0.1793,  0.1317, -0.2021,
          0.2467,  0.1251, -0.0857,  0.1589, -0.0829, -0.1605,  0.1649, -0.2431,
         -0.1792,  0.0058, -0.2280, -0.1048,  0.1146, -0.2707,  0.2562,  0.2163,
          0.1671,  0.1597, -0.2725, -0.0851, -0.1821, -0.0893,  0.1568,  0.1680],
        [-0.1372,  0.2515,  0.1529,  0.1607, -0.1567, -0.1800,  0.1103, -0.1928,
          0.2575,  0.1267, -0.0851,  0.1515, -0.0770, -0.1579,  0.1576, -0.2630,
         -0.1682,  0.0060, -0.2539, -0.1088,  0.1103, -0.2712,  0.2430,  0.2114,
          0.1470,  0.1720, -0.2743, -0.0672, -0.1609, -0.1009,  0.1836,  0.1737]],
       device='cuda:0', grad_fn=<DivBackward0>)